## Regresión Logística multiclase

## Cargar librerías

In [1]:
import sys
sys.path.insert(0, r'C:\Users\Administrator\Documents\Proyecto seg clientes refuerzo\myenv\Lib\site-packages')
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import optuna
from sklearn.model_selection import cross_val_score


C:\Users\Administrator\Documents\Proyecto seg clientes refuerzo\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Carga de datos

In [2]:
# Cargar el dataset
file_path = r'C:/Users/Administrator/Documents/Proyecto seg clientes refuerzo/Proyecto_seg_clientes/Data/teleCust1000t_listo.csv'
df = pd.read_csv(file_path)

# Ver las primeras filas para confirmar la carga
print(df.head())

   region  tenure  age  marital  address  income  ed  employ  retire  gender  \
0       2      13   44        1        9    64.0   4       5       0       0   
1       3      11   33        1        7   136.0   5       5       0       0   
2       3      68   52        1       24   116.0   1      29       0       1   
3       2      33   33        0       12    33.0   2       0       0       1   
4       2      23   30        1        9    30.0   1       2       0       0   

   reside  custcat  
0       2        1  
1       6        4  
2       2        3  
3       1        1  
4       4        3  


## Separar características y etiqueta

Identifica la variable objetivo (custcat) y separa las columnas de características (independientes) de la columna objetivo (dependiente).

In [3]:
# Separar características (X) y variable objetivo (y)
X = df.drop(columns=['custcat'])  # Características
y = df['custcat']  # Variable objetivo

## Dividir los datos en entrenamiento y prueba

Usa train_test_split para dividir los datos en conjuntos de entrenamiento y prueba. Esto permite evaluar el modelo después del entrenamiento.

In [4]:
# Dividir los datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Escalar las características

Usamos StandardScaler para normalizar las características numéricas, lo que asegura que todas las variables estén en una escala comparable.

In [5]:
# Escalar las características
scaler = StandardScaler()

# Ajustar el escalador en los datos de entrenamiento y transformarlos
X_train_scaled = scaler.fit_transform(X_train)

# Transformar los datos de prueba con el mismo escalador
X_test_scaled = scaler.transform(X_test)

## Crear y entrenar el modelo (regresión logística multiclase)

Creamos el modelo de regresión logística multiclase y lo entrenamos con los datos escalados.

In [6]:
# Crear el modelo de regresión logística
model = LogisticRegression(multi_class='ovr', max_iter=1000, random_state=42)

# Entrenar el modelo
model.fit(X_train_scaled, y_train)


C:\Users\Administrator\Documents\Proyecto seg clientes refuerzo\myenv\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='ovr', random_state=42)

## Evaluar el modelo

Usamos el conjunto de prueba para evaluar el modelo.

In [7]:
# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test_scaled)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy:.2f}')


Precisión del modelo: 0.44


In [8]:
# Exactitud del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Exactitud del modelo: {accuracy:.2f}')

# Informe de clasificación
print('Informe de clasificación:')
print(classification_report(y_test, y_pred))

Exactitud del modelo: 0.44
Informe de clasificación:
              precision    recall  f1-score   support

           1       0.46      0.43      0.44        60
           2       0.37      0.33      0.35        39
           3       0.48      0.51      0.50        55
           4       0.42      0.46      0.44        46

    accuracy                           0.44       200
   macro avg       0.43      0.43      0.43       200
weighted avg       0.44      0.44      0.44       200



In [9]:
# Definir la función objetivo
def objective(trial):
    # Sugerir valores para los hiperparámetros
    C = trial.suggest_loguniform('C', 1e-4, 1e2)  # Regularización
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    
    # Si el solver no soporta un tipo de penalización, lo ignoramos
    if solver == 'liblinear' and penalty == 'l1':
        penalty = 'l1'
    elif solver == 'liblinear' and penalty == 'l2':
        penalty = 'l2'
    elif solver == 'saga' and penalty not in ['l1', 'l2']:
        return float('inf')  # Penalización no válida
    
    # Crear el modelo con los hiperparámetros sugeridos
    model = LogisticRegression(C=C, solver=solver, penalty=penalty, max_iter=1000, random_state=42)
    
    # Evaluar el modelo usando validación cruzada
    scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')
    return scores.mean()  # Devolver la métrica de evaluación


In [10]:
# Crear el estudio
study = optuna.create_study(direction='maximize')  # Maximizar la precisión

# Ejecutar la optimización
study.optimize(objective, n_trials=50)  # Número de iteraciones

# Mostrar los mejores resultados
print('Número de la mejor prueba:', study.best_trial.number)
print('Mejores hiperparámetros:', study.best_trial.params)
print('Mejor precisión:', study.best_trial.value)


[I 2024-12-06 11:53:13,689] A new study created in memory with name: no-name-bb3a2ca6-c806-4f4b-aa5d-70712b7e8a41
C:\Users\Administrator\AppData\Local\Temp\ipykernel_7784\3430117858.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-4, 1e2)  # Regularización
[I 2024-12-06 11:53:14,057] Trial 0 finished with value: 0.28250000000000003 and parameters: {'C': 0.00095225512965523, 'solver': 'saga', 'penalty': 'l1'}. Best is trial 0 with value: 0.28250000000000003.
C:\Users\Administrator\AppData\Local\Temp\ipykernel_7784\3430117858.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-4, 1e2)  # Regulari

Número de la mejor prueba: 25
Mejores hiperparámetros: {'C': 0.04286628928184899, 'solver': 'saga', 'penalty': 'l1'}
Mejor precisión: 0.41500000000000004


In [11]:
# Obtener los mejores hiperparámetros
best_params = study.best_trial.params

# Crear el modelo final
final_model = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    penalty=best_params['penalty'],
    max_iter=1000,
    random_state=42
)

# Entrenar el modelo con el conjunto completo de entrenamiento
final_model.fit(X_train_scaled, y_train)

# Evaluar el modelo en el conjunto de prueba
y_pred = final_model.predict(X_test_scaled)
final_accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión final del modelo: {final_accuracy:.2f}')
print('Informe de clasificación:')
print(classification_report(y_test, y_pred))


Precisión final del modelo: 0.40
Informe de clasificación:
              precision    recall  f1-score   support

           1       0.48      0.48      0.48        60
           2       0.06      0.03      0.04        39
           3       0.45      0.55      0.50        55
           4       0.33      0.41      0.37        46

    accuracy                           0.40       200
   macro avg       0.33      0.37      0.35       200
weighted avg       0.36      0.40      0.37       200

